In [81]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
import sys
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/tree-prompt/aug22_vary_train_test'

r = imodelsx.process_results.get_results_df(results_dir)
experiment_filename = '../experiments/01_fit.py'
r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename)
# # r['mean_llm_calls'] = notebook_helper.add_mean_llm_calls(r)
# r = r[r.model_name != 'manual_rf']
# r = r[r.checkpoint == 'gpt2']
# r = r[r.model_name != 'manual_tree']
# r.to_pickle('../results/subsample_results.pkl')

# r = pd.read_pickle('../results/subsample_results.pkl')
# ravg = r[r.seed == 1]
# ravg = imodelsx.process_results.average_over_seeds(r, experiment_filename='../experiments/01_fit.py')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 60/60 [00:04<00:00, 14.56it/s]


In [83]:
metric = 'accuracy_test'
r['suffix'] = ''
r.loc[r['num_prompts'] == 1, 'model_name'] = 'single prompt'
ks = ["dataset_name", "checkpoint", "checkpoint_evaluation", "model_name"]
r2 = r[ks + [metric]].sort_values(by=ks)

In [84]:
# display all rows
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(r2)

,dataset_name,checkpoint,checkpoint_evaluation,model_name,accuracy_test
59,emotion,EleutherAI/gpt-j-6B,EleutherAI/gpt-j-6B,manual_gbdt,0.543000
51,emotion,EleutherAI/gpt-j-6B,EleutherAI/gpt-j-6B,manual_tree,0.516000
58,emotion,EleutherAI/gpt-j-6B,EleutherAI/gpt-j-6B,single prompt,0.409500
30,emotion,EleutherAI/gpt-j-6B,gpt2,manual_gbdt,0.369500
11,emotion,EleutherAI/gpt-j-6B,gpt2,manual_tree,0.454000
19,emotion,EleutherAI/gpt-j-6B,gpt2,single prompt,0.374500
54,emotion,gpt2,EleutherAI/gpt-j-6B,manual_gbdt,0.353500
13,emotion,gpt2,EleutherAI/gpt-j-6B,manual_tree,0.424500
56,emotion,gpt2,EleutherAI/gpt-j-6B,single prompt,0.430000
47,emotion,gpt2,gpt2,manual_gbdt,0.557500


In [85]:
means = r2.groupby([k for k in ks if not k == 'dataset_name'])[metric].mean().reset_index()

In [86]:
means.pivot_table(
    index=["checkpoint", "model_name"], columns="checkpoint_evaluation", values=metric
).style.format("{:.2f}").background_gradient(cmap='Blues')